# LMQL in Python

In [1]:
# setup lmql path (not shown in documentation, metadata has nbshpinx: hidden)
import sys 
sys.path.append("../../../src/")
# load and set OPENAI_API_KEY
import os 
os.environ["OPENAI_API_KEY"] = open("../../../api.env").read().split("\n")[1].split(": ")[1].strip()

%load_ext autoreload
%autoreload 2

# disable logit bias logging
import lmql.runtime.bopenai.batched_openai as batched_openai
batched_openai.set_logit_bias_logging(False)

LMQL can be used in the Playground or as a standalone language. However, it is also possible to use LMQL as a Python library. This allows you to integrate LMQL code into your own Python projects. This chapter provides a brief overview of the LMQL Python Integration.

### LMQL as Decorated Python Functions

To define and run LMQL queries within Python, you can simply decorate a Python function with the `lmql.query` decorator, and provide the query code as a multi-line string. The decorated function will then automatically be compiled into a LMQL query, and can be called with the same arguments as the original Python function. The return value of the decorated function will be the result of the LMQL query.

In [4]:
import lmql

@lmql.query
async def hello(): 
    '''lmql
    argmax 
        "Hello[WHO]" 
    from 
        "openai/text-ada-001" 
    where 
        len(WHO) < 10
    '''

(await hello())[0].prompt

'Hello everyone'

**Asynchronous API** LMQL implements a fully asynchronous API. This means that all LMQL queries are executed asynchronously, and return a `Future` object. To execute a query, you therefore have to use the `await` keyword as shown above. By relying on the `asyncio` library, LMQL also supports the execution of arbitrary many queries in parallel, using `asyncio.gather`, where on the backend the runtime will automatically optimize and schedule the queries to maximize throughput (e.g. auto batching across multiple queries).

**Result Type** An LMQL query that is executed in a Python context, always returns a list of `LMQLResult` objects, where the number of elements is determined by the underlying decoding algorithm (e.g. `n` with `beam(n=n)`). The `LMQLResult` object is a simple dataclass with the following fields:

```python
class LMQLResult:
    # full prompt with all variables substituted
    prompt: str 
    # a dictionary of all assigned template variable values
    variables: Dict[str, str] 
```

From such a result object, you can easily extract the prompt and the assigned variable values, no need for further processing. For `distribution` variables (cf. [Distribution Clause](../language/overview.md#extracting-more-information-with-distributions)), the `variables` dictionary will additionally contain `log P(VAR)` and `P(VAR)` entries, allowing users to easily extract the probability of the assigned value.

`@lmql.query` functions like `hello()` also conform to the interface required by the `langchain` library, as detailed in the [LangChain Integration](./langchain.ipynb#Using-LMQL-from-LangChain) chapter.

### Program Context

`@lmql.query` annotated functions are fully integrated into the provided program context, meaning you can provide function arguments and even access variables from the containing scope during query execution. This allows you to use LMQL as a fully integrated part of your Python program:

In [3]:
import lmql

someone = "the Python interpreter"

@lmql.query
async def greet(someone_else):
    '''
    argmax "Greet {someone} and {someone_else}: [WHO]" from "openai/text-ada-001" where len(WHO) < 20
    '''

(await greet("the World"))[0].prompt

'Greet the Python interpreter and the World: \n\nHello, I am here!'

This also allows you to call utility functions, that query external information systems like Wikipedia, and incorporate the obtained information dynamically in your prompt:

In [7]:
import lmql
import aiohttp

async def look_up(term):
    # looks up term on wikipedia
    url = f"https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles={term}&origin=*"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            # get the first sentence on first page
            page = (await response.json())["query"]["pages"]
            return list(page.values())[0]["extract"].split(".")[0]

@lmql.query
async def greet(term):
    '''
    argmax 
        """Greet {term} ({await look_up(term)}): 
        Hello[WHO]
        """
    from 
        "openai/text-davinci-003" 
    where 
        STOPS_AT(WHO, "\n")
    '''

print((await greet("Earth"))[0].prompt)

Greet Earth (Earth is the third planet from the Sun and the only place known in the universe where life has originated and found habitability): 
Hello Earth! It's wonderful to meet you. You are a unique and special place in the universe, and we are so lucky to call you home.



This capability of calling arbitrary program logic during query execution, enables powerful use cases like the integration of retrieval, as discussed in the [LangChain Integration](./langchain.ipynb) chapter.

### Syntax Highlighting

LMQL also provides a syntax highlighter for Visual Studio Code. Both `.lmql` files as well as `.py` files with `@lmql.query` decorated functions will be highlighted automatically. The syntax highlighter is available as a [VSCode extension](https://marketplace.visualstudio.com/items?itemName=lmql-team.lmql).

To enable syntax highlighting in Python files, make sure to lead the LMQL code in an `@lmql.query` function with the `lmql` code block type:

<img width="528" alt="image" src="https://user-images.githubusercontent.com/17903049/230897738-a676cd6b-bec2-4403-8628-1c10d3da263c.png">
